In [1]:

!pip install -q sentence-transformers transformers "faiss-cpu>=1.7.3" pandas langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 25.4 MB/s eta 0:00:00


In [2]:
# CELL 1
import os
import re
import json
import numpy as np
import pandas as pd
import faiss
import torch
from langdetect import detect
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MarianTokenizer, MarianMTModel

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)


Device: cuda


In [3]:
# CELL 3
df = pd.read_csv("/content/ipc_sections.csv")
print("Rows:", len(df))
print("Columns:", df.columns.tolist())
display(df.head(6))


Rows: 444
Columns: ['Description', 'Offense', 'Punishment', 'Section']


,Description,Offense,Punishment,Section
0,Description of IPC Section 140\nAccording to s...,Wearing the dress or carrying any token used b...,3 Months or Fine or Both,IPC_140
1,Description of IPC Section 127\nAccording to s...,Receiving property taken by war or depredation...,7 Years + Fine + forfeiture of property,IPC_127
2,Description of IPC Section 128\nAccording to s...,Public servant voluntarily allowing prisoner o...,Imprisonment for Life or 10 Years + Fine,IPC_128
3,Description of IPC Section 129\nAccording to s...,Public servant negligently suffering prisoner ...,Simple Imprisonment 3 Years + Fine,IPC_129
4,Description of IPC Section 130\nAccording to s...,"Aiding escape of, rescuing or harbouring, such...",Imprisonment for Life or 10 Years + Fine,IPC_130
5,Description of IPC Section 131\nAccording to s...,"Abetting mutiny, or attempting to seduce an of...",Imprisonment for Life or 10 Years + Fine,IPC_131


In [4]:
# REPLACEMENT for CELL 4 — explicitly set column names and normalize section

col_section = "Section"
col_offense = "Offense"
col_description = "Description"
col_punishment = "Punishment"

print("Using columns:")
print(" section:", col_section)
print(" offense:", col_offense)
print(" description:", col_description)
print(" punishment:", col_punishment)

# Normalization function (safe)
import re
import pandas as pd

def normalize_section(s):
    if pd.isna(s):
        return None
    s = str(s).strip()
    s = re.sub(r'(?i)ipc[_\s]*', '', s)
    s = re.sub(r'(?i)section\s*', '', s)
    s = re.sub(r'[\s\-]+', '', s)
    return s.upper()


if col_section in df.columns:
    df['section_norm'] = df[col_section].apply(normalize_section)
else:
    df['section_norm'] = None

# Show preview
print("\nPreview (first 8 rows):")
display(df[[col_section, 'section_norm', col_offense, col_punishment]].head(8))



Using columns:
 section: Section
 offense: Offense
 description: Description
 punishment: Punishment

Preview (first 8 rows):


,Section,section_norm,Offense,Punishment
0,IPC_140,140,Wearing the dress or carrying any token used b...,3 Months or Fine or Both
1,IPC_127,127,Receiving property taken by war or depredation...,7 Years + Fine + forfeiture of property
2,IPC_128,128,Public servant voluntarily allowing prisoner o...,Imprisonment for Life or 10 Years + Fine
3,IPC_129,129,Public servant negligently suffering prisoner ...,Simple Imprisonment 3 Years + Fine
4,IPC_130,130,"Aiding escape of, rescuing or harbouring, such...",Imprisonment for Life or 10 Years + Fine
5,IPC_131,131,"Abetting mutiny, or attempting to seduce an of...",Imprisonment for Life or 10 Years + Fine
6,IPC_132,132,"Abetment of mutiny, if mutiny is committed in ...",Death or Imprisonment for Life or 10 Years + Fine
7,IPC_133,133,"Abetment of an assault by An officer, soldier,...",3 Years + Fine


In [5]:
csv_path=df

In [6]:
# Replacement CELL 5
import os, re, pandas as pd


if 'df' not in globals():
    raise RuntimeError("DataFrame 'df' not found. Run the CSV load cell first (CELL 3).")


if 'csv_path' in globals() and isinstance(csv_path, str):
    source_name = os.path.basename(csv_path)
else:

    source_name = "uploaded_csv"


col_section = "Section"
col_offense = "Offense"
col_description = "Description"
col_punishment = "Punishment"


for c in df.columns:
    lc = c.lower().strip()
    if lc == 'section':
        col_section = c
    if 'offenc' in lc:
        col_offense = c
    if 'description' in lc or 'in-depth' in lc or 'explan' in lc:
        col_description = c
    if 'punish' in lc:
        col_punishment = c

print("Using columns:", col_section, col_offense, col_description, col_punishment)
print("Source name:", source_name)


def normalize_section(s):
    if pd.isna(s):
        return None
    s = str(s).strip()
    s = re.sub(r'(?i)ipc[_\s]*', '', s)
    s = re.sub(r'(?i)section\s*', '', s)
    s = re.sub(r'[\s\-]+', '', s)
    return s.upper()


if col_section in df.columns:
    df['section_norm'] = df[col_section].apply(normalize_section)
else:
    df['section_norm'] = None


chunks = []
for i, row in df.iterrows():
    sec = row.get('section_norm') if 'section_norm' in row.index else None
    parts = []
    if sec:
        parts.append(f"Section {sec}")
    if col_offense in row.index and pd.notna(row[col_offense]):
        parts.append(str(row[col_offense]).strip())
    if col_description in row.index and pd.notna(row[col_description]):
        parts.append(str(row[col_description]).strip())
    if col_punishment in row.index and pd.notna(row[col_punishment]):
        parts.append("Punishment: " + str(row[col_punishment]).strip())
    passage = " ".join(parts).strip()
    if not passage:
        continue
    meta = {"row_id": int(i), "section": sec, "source": source_name}
    chunks.append({"text": passage, "meta": meta})

print("Created chunks:", len(chunks))
# preview a few
for x in chunks[:5]:
    print(x['meta'])
    print(x['text'][:400])
    print("---")



Using columns: Section Offense Description Punishment
Source name: uploaded_csv
Created chunks: 444
{'row_id': 0, 'section': '140', 'source': 'uploaded_csv'}
Section 140 Wearing the dress or carrying any token used by a soldier, sailor or airman with intent that it may be believed that he is such a soldier, sailor or airman Description of IPC Section 140
According to section 140 of Indian penal code, Whoever, not being a soldier, sailor or airman in the Military, Naval or Air service of the Government of India, wears any garb or carries any token resem
---
{'row_id': 1, 'section': '127', 'source': 'uploaded_csv'}
Section 127 Receiving property taken by war or depredation mentioned in sections 125 And 126 Description of IPC Section 127
According to section 127 of Indian penal code, Whoever receives any property knowing the same to have been taken in the commission of any of the offences mentioned in sections 125 and 126, shall be punished with imprisonment of either description for a te

In [7]:
# CELL 6
EMB_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
print("Loading embedding model:", EMB_MODEL_NAME)
embed_model = SentenceTransformer(EMB_MODEL_NAME, device=DEVICE)

texts = [c["text"] for c in chunks]
print("Embedding passages (this can take a few minutes)...")
embeddings = embed_model.encode(texts, batch_size=32, show_progress_bar=True, convert_to_numpy=True)
embeddings = np.asarray(embeddings).astype('float32')  # ensure float32

print("Embeddings shape:", embeddings.shape)
d = embeddings.shape[1]
faiss.normalize_L2(embeddings)
index = faiss.IndexFlatIP(d)
index.add(embeddings)
print("FAISS index built, ntotal:", index.ntotal)


Loading embedding model: sentence-transformers/paraphrase-multilingual-mpnet-base-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding passages (this can take a few minutes)...


Batches:   0%|          | 0/14 [00:00<?, ?it/s]

Embeddings shape: (444, 768)
FAISS index built, ntotal: 444


In [8]:
# CELL 6
EMB_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
print("Loading embedding model:", EMB_MODEL_NAME)
embed_model = SentenceTransformer(EMB_MODEL_NAME, device=DEVICE)

texts = [c["text"] for c in chunks]
print("Embedding passages (this can take a few minutes)...")
embeddings = embed_model.encode(texts, batch_size=32, show_progress_bar=True, convert_to_numpy=True)
embeddings = np.asarray(embeddings).astype('float32')  # ensure float32

print("Embeddings shape:", embeddings.shape)
d = embeddings.shape[1]
faiss.normalize_L2(embeddings)
index = faiss.IndexFlatIP(d)
index.add(embeddings)
print("FAISS index built, ntotal:", index.ntotal)


Loading embedding model: sentence-transformers/paraphrase-multilingual-mpnet-base-v2
Embedding passages (this can take a few minutes)...


Batches:   0%|          | 0/14 [00:00<?, ?it/s]

Embeddings shape: (444, 768)
FAISS index built, ntotal: 444


In [9]:
# CELL 7
ART_DIR = "rag_csv_artifacts"
os.makedirs(ART_DIR, exist_ok=True)

chunks_path = os.path.join(ART_DIR, "chunks.json")
with open(chunks_path, "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)
print("Saved chunks ->", chunks_path)

emb_path = os.path.join(ART_DIR, "embeddings.npy")
np.save(emb_path, embeddings)
print("Saved embeddings ->", emb_path)

faiss_path = os.path.join(ART_DIR, "faiss.index")
faiss.write_index(index, faiss_path)
print("Saved faiss index ->", faiss_path)

# Colab download if available
try:
    from google.colab import files as colab_files
    colab_files.download(chunks_path)
    colab_files.download(emb_path)
    colab_files.download(faiss_path)
except Exception:
    print("Not in Colab (or download blocked). Artifacts saved to", os.path.abspath(ART_DIR))


Saved chunks -> rag_csv_artifacts/chunks.json
Saved embeddings -> rag_csv_artifacts/embeddings.npy
Saved faiss index -> rag_csv_artifacts/faiss.index


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
# CELL 8
# Marian translators (English <-> Hindi)
print("Loading translation models (Marian)...")
en_hi_tok = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
en_hi_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-hi").to(DEVICE)
hi_en_tok = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-hi-en")
hi_en_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-hi-en").to(DEVICE)

def translate_en_to_hi(text, max_len=512):
    tokens = en_hi_tok([text], return_tensors="pt", truncation=True, max_length=max_len).to(DEVICE)
    with torch.no_grad():
        out = en_hi_model.generate(**tokens, num_beams=4, early_stopping=True, max_length=max_len+50)
    return en_hi_tok.batch_decode(out, skip_special_tokens=True)[0]

# Generator: instruction-tuned model (Flan-T5 gives fewer sentinel artifacts)
GEN_MODEL_NAME = "google/flan-t5-small"
print("Loading generator:", GEN_MODEL_NAME)
gen_tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL_NAME)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(GEN_MODEL_NAME).to(DEVICE)

def generate_answer(prompt, max_new_tokens=180):
    inputs = gen_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(DEVICE)
    with torch.no_grad():
        out = gen_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=3
        )
    txt = gen_tokenizer.batch_decode(out, skip_special_tokens=True)[0]
    txt = re.sub(r"<extra_id_\d+>", "", txt).strip()
    return txt


Loading translation models (Marian)...


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:


import re
from langdetect import detect

# 1) retrieval (same as before)
def retrieve_chunks_from_csv(question, top_k=6):
    if 'index' not in globals() or 'chunks' not in globals():
        raise RuntimeError("FAISS index or chunks not found. Run embedding / chunk creation cells first.")
    q_emb = embed_model.encode([question], convert_to_numpy=True)
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, top_k)
    results = []
    for rank, idx in enumerate(I[0]):
        if idx < 0 or idx >= len(chunks):
            continue
        c = chunks[idx]
        results.append({
            "rank": rank+1,
            "chunk_id": c["meta"]["row_id"],
            "page_no": None,
            "sections": [c["meta"].get("section")] if c["meta"].get("section") else [],
            "language": "en",
            "orig_text": c["text"],
            "score": float(D[0][rank])
        })
    return results

def get_exact_quote_for_section(section_label, max_words=30, return_full=False):
    """
    Returns:
      exact_quote (<=max_words string), meta, punishment_text (full or None), offense_text (full or None), full_text (full chunk text)
    If not found: returns (None, None, None, None, None)
    return_full: if True, exact_quote will be the full offense text (not truncated).
    """
    if not section_label:
        return None, None, None, None, None
    normalized = re.sub(r'[\s\-_]+', '', str(section_label)).upper()
    for c in chunks:
        sec = c["meta"].get("section")
        if sec and sec.upper() == normalized:
            full_text = c["text"]
            # extract punishment verbatim if present
            pun = None
            off = None
            try:
                if "Punishment:" in full_text:
                    head, pun_part = full_text.split("Punishment:", 1)
                    pun = pun_part.strip()
                else:
                    m = re.search(r'punishment[:\s]+(.+)', full_text, flags=re.IGNORECASE)
                    if m:
                        pun = m.group(1).strip()
                # offense: text after "Section <sec>"
                head_text = re.sub(r'(?i)^Section\s*' + re.escape(sec), '', full_text, count=1).strip()
                if "Punishment:" in head_text:
                    off = head_text.split("Punishment:", 1)[0].strip()
                else:
                    off = head_text.strip()
            except Exception:
                pun = None
                off = None

            words = full_text.split()
            if return_full:
                exact_quote = off  # full offense text
            else:
                exact_quote = " ".join(words[:max_words])
            return exact_quote, c["meta"], pun, off, full_text
    return None, None, None, None, None

# 3) adaptive prompt builder (detects punishment/exact quote intent)
def build_prompt_csv(question, retrieved, answer_lang="auto", max_quote_words=40, include_top_k=2, snippet_words=120):
    try:
        q_lang = detect(question)
    except:
        q_lang = "en"
    if answer_lang == "auto":
        answer_lang = q_lang

    q_lower = question.lower()
    asks_punishment = any(k in q_lower for k in [
        "punishment", "sentence", "imprison", "fine", "years", "jail", "death",
        "सजा", "दंड", "जमानत", "जेल", "सज़ा", "सजा"
    ])
    asks_exact_quote = any(k in q_lower for k in ["quote", "quotation", "exact quote", "≤", "<=", "≤"])

    if asks_punishment:
        instruction_header = (
            "User asked about punishment. Use the retrieved passages and provide:\n"
            "1) Full punishment EXACTLY as written in the dataset (verbatim).\n"
            "2) A short offense summary (1-2 sentences).\n"
            "3) Provide citation ROW_ID and SECTION.\n"
            "4) Optionally include a short quoted excerpt (<= {} words) if requested.\n"
        ).format(max_quote_words)
    elif asks_exact_quote:
        instruction_header = (
            "User requested an exact quote. Provide the exact English excerpt (<= {} words) from the retrieved passage, then translate if requested. "
            "Cite ROW_ID / SECTION.\n"
        ).format(max_quote_words)
    else:
        instruction_header = (
            "Provide a concise answer (1-3 sentences). If the user asked about punishment, include the punishment verbatim. "
            "Cite ROW_ID or SECTION for factual claims.\n"
        )

    prompt = (
        "You are a careful legal assistant. Use ONLY the retrieved passages below to answer the user's question.\n"
        + instruction_header +
        "End with: 'Not legal advice — consult a qualified lawyer.'\n\n"
    )
    prompt += f"User question: {question}\nRequested language: {answer_lang}\n\n"
    prompt += "Retrieved passages (original English):\n"
    for r in retrieved[:include_top_k]:
        sec_list = r.get("sections", [])
        sec_str = sec_list[0] if sec_list else "[]"
        snippet = " ".join(r["orig_text"].split()[:snippet_words]).replace("\n", " ")
        prompt += f"--- ROW_ID:{r['chunk_id']} SECTION:{sec_str} SCORE:{r['score']:.3f} ---\n"
        prompt += snippet + "\n\n"

    prompt += (
        "\nInstructions summary:\n"
        "1) Use retrieved passages only. 2) Cite ROW_ID/SECTION. 3) If user asked punishment, print punishment verbatim. "
        "4) If exact quote requested, obey length <= {} words; otherwise include full offense/punishment as needed.\n"
    ).format(max_quote_words)
    return prompt, answer_lang

# 4) main rag function (returns full answer unless exact short quote requested)
def rag_answer_csv(question, answer_lang="auto", top_k=8, include_top_k=2, snippet_words=120, max_quote_words=40, debug=True):
    # detect explicit section mention
    sec_match = re.search(r'\bsection\s*([0-9]+[A-Za-z]?)\b', question, flags=re.IGNORECASE)
    if not sec_match:
        sec_match = re.search(r'धारा\s*([0-9]+[A-Za-z]?)', question, flags=re.IGNORECASE)

    q_lower = question.lower()
    asks_punishment = any(k in q_lower for k in [
        "punishment", "sentence", "imprison", "fine", "years", "jail",
        "सजा", "दंड", "जेल", "सज़ा", "सजा"
    ])
    asks_exact_quote = any(k in q_lower for k in ["quote", "quotation", "exact quote", "≤", "<=", "≤", "उद्धरण", "सटीक उद्धरण"])

    # If explicit section found, return deterministic, full info (unless user requested a short exact quote)
    if sec_match:
        sec_label = sec_match.group(1)
        exact_quote, meta, punishment_text, offense_text, full_text = get_exact_quote_for_section(sec_label, max_words=max_quote_words, return_full=False)
        # If user asked for exact short quote -> return <= max_quote_words
        if asks_exact_quote and exact_quote:
            quote_hi = translate_en_to_hi(exact_quote) if answer_lang.startswith("hi") else None
            answer_text = f"Exact quote (EN):\n\"{exact_quote}\"\n\nSource: ROW_ID {meta['row_id']} SECTION {meta.get('section')}\n\n"
            if quote_hi:
                answer_text += f"Hindi translation:\n\"{quote_hi}\"\n\n"
            answer_text += "Not legal advice — consult a qualified lawyer."
            return {"question": question, "answer_lang": "hi" if answer_lang.startswith("hi") else "en", "generated_raw": answer_text, "final_answer": answer_text, "retrieved": [], "citations":[{"row_id":meta['row_id'],"section":meta.get('section'),"quote":exact_quote,"punishment":punishment_text}], "prompt_used": None}

        # Otherwise return full offense + full punishment (if present)
        if meta:
            parts = []
            if offense_text:
                parts.append("Offense (full):\n" + offense_text.strip())
            if punishment_text:
                parts.append("\nPunishment (verbatim):\n" + punishment_text.strip())
            # include full chunk as source for transparency
            parts.append(f"\nSource: ROW_ID {meta['row_id']} SECTION {meta.get('section')}")
            parts.append("\nNot legal advice — consult a qualified lawyer.")
            final = "\n\n".join(parts)
            return {"question": question, "answer_lang":"hi" if answer_lang.startswith("hi") else "en", "generated_raw": final, "final_answer": final, "retrieved": [], "citations":[{"row_id":meta['row_id'],"section":meta.get('section'),"quote":exact_quote,"punishment":punishment_text}], "prompt_used": None}

    # Standard RAG path (no explicit section or not found)
    retrieved = retrieve_chunks_from_csv(question, top_k=top_k)
    prompt, chosen_answer_lang = build_prompt_csv(question, retrieved, answer_lang=answer_lang, max_quote_words=max_quote_words, include_top_k=include_top_k, snippet_words=snippet_words)

    if debug:
        print("Prompt length (chars):", len(prompt))
        for r in retrieved[:min(6,len(retrieved))]:
            print(f" ROW {r['chunk_id']} SEC {r['sections']} SCORE {r['score']:.3f}")
            print("  snippet:", r['orig_text'][:160].replace('\\n',' '), "...\n")

    gen_text = generate_answer(prompt, max_new_tokens=300)
    final_answer = gen_text

    # translate whole answer if user asked Hindi
    if answer_lang.startswith("hi"):
        try:
            gen_lang = detect(gen_text)
        except:
            gen_lang = "en"
        if gen_lang != "hi":
            try:
                final_answer = translate_en_to_hi(gen_text)
            except Exception:
                final_answer = gen_text + "\n\n(Translation failed.)"

    # prepare citations with punishment extraction if present
    citations = []
    for r in retrieved:
        pun = None
        m = re.search(r'Punishment[:\s]+(.+)', r['orig_text'], flags=re.IGNORECASE)
        if m:
            pun = m.group(1).strip()
        quote = None
        if asks_exact_quote:
            quote = " ".join(r["orig_text"].split()[:max_quote_words])
            translation = translate_en_to_hi(quote) if answer_lang.startswith("hi") else None
        else:
            quote = r["orig_text"]  # full text as quote
            translation = translate_en_to_hi(quote) if answer_lang.startswith("hi") else None
        citations.append({"row_id": r["chunk_id"], "section": r["sections"][0] if r["sections"] else None, "score": r["score"], "quote": quote, "translation": translation, "punishment": pun})

    return {"question": question, "answer_lang": chosen_answer_lang, "generated_raw": gen_text, "final_answer": final_answer, "retrieved": retrieved, "citations": citations, "prompt_used": prompt}

# ---------------------------
# Quick tests (run after cell)
# ---------------------------
print("Definitions loaded. Test examples:")




Definitions loaded. Test examples:


In [14]:
# CELL 11
print("Interactive CSV-RAG loop. Type 'exit' to stop.")
while True:
    q = input("\nEnter your question (Hindi or English): ").strip()
    if q.lower() in ["exit", "quit"]:
        break
    al = input("Requested answer language ('hi'/'en'/'auto', default auto): ").strip().lower() or "auto"
    out = rag_answer_csv(q, answer_lang=al, top_k=8, include_top_k=2, snippet_words=120, debug=True)
    print("\n--- FINAL ANSWER ---\n")
    print(out["final_answer"])
    print("\n--- CITATIONS ---")
    for c in out["citations"][:6]:
        print(c)
    # option to save current artifacts again if needed
    s = input("\nSave artifacts now? (y/n) ").strip().lower()
    if s == 'y':
        # re-save embeddings and index (useful if you re-embed)
        np.save(emb_path, embeddings)
        faiss.write_index(index, faiss_path)
        with open(chunks_path, "w", encoding="utf-8") as f:
            json.dump(chunks, f, ensure_ascii=False, indent=2)
        try:
            from google.colab import files as colab_files
            colab_files.download(chunks_path)
            colab_files.download(emb_path)
            colab_files.download(faiss_path)
        except Exception:
            print("Artifacts saved in", os.path.abspath(ART_DIR))


Interactive CSV-RAG loop. Type 'exit' to stop.

Enter your question (Hindi or English): what does section 140 IPC state?
Requested answer language ('hi'/'en'/'auto', default auto): hi

--- FINAL ANSWER ---

Offense (full):
Wearing the dress or carrying any token used by a soldier, sailor or airman with intent that it may be believed that he is such a soldier, sailor or airman Description of IPC Section 140
According to section 140 of Indian penal code, Whoever, not being a soldier, sailor or airman in the Military, Naval or Air service of the Government of India, wears any garb or carries any token resembling any garb or token used by such a soldier, sailor or airman with the intention that it may be believed that he is such a soldier, sailor or airman, shall be punished with imprisonment of either description for a term which may extend to three months, or with fine which may extend to five hundred rupees, or with both.


IPC 140 in Simple Words
If someone who is not a military member

In [15]:
!pip freeze > requirements.txt
